In [9]:
import joblib
import numpy as np
from dataread import *
from DATA_INDICATORS import *
from sklearn.preprocessing import MinMaxScaler
import pymysql
from datetime import datetime
host = "ai-predict.cyyrt9e5hsme.ap-northeast-2.rds.amazonaws.com"
port = 3306
username = "admin"
database = "AI_predict"
password = "825582qaz"

In [10]:
def connect_RDS(host,port,username,password,database):
    conn = pymysql.connect(host = host,user = username, passwd=password, db=database,
    port=port, use_unicode = True, charset='utf8')
    return conn

In [11]:
def make_dataset(data, label, window_size=5):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [18]:
ticker = download_data().Usdt_symbol()
for code_name in ticker:
    conn = connect_RDS(host,port,username,password,database)
    cursor = conn.cursor()
    print(code_name)
    # 테이블 생성
    sql = '''CREATE TABLE ''' + str(code_name).replace("/","_") + '''( 
date DATE, 
predict varchar(255)
)
    '''
    print(sql)
    cursor.execute(sql)
    conn.commit()
    conn.close()

BTC/USDT
CREATE TABLE BTC_USDT( 
date DATE, 
predict varchar(255)
)
    
ETH/USDT
CREATE TABLE ETH_USDT( 
date DATE, 
predict varchar(255)
)
    
BNB/USDT
CREATE TABLE BNB_USDT( 
date DATE, 
predict varchar(255)
)
    
BCC/USDT
CREATE TABLE BCC_USDT( 
date DATE, 
predict varchar(255)
)
    
NEO/USDT
CREATE TABLE NEO_USDT( 
date DATE, 
predict varchar(255)
)
    
LTC/USDT
CREATE TABLE LTC_USDT( 
date DATE, 
predict varchar(255)
)
    
QTUM/USDT
CREATE TABLE QTUM_USDT( 
date DATE, 
predict varchar(255)
)
    
ADA/USDT
CREATE TABLE ADA_USDT( 
date DATE, 
predict varchar(255)
)
    
XRP/USDT
CREATE TABLE XRP_USDT( 
date DATE, 
predict varchar(255)
)
    
EOS/USDT
CREATE TABLE EOS_USDT( 
date DATE, 
predict varchar(255)
)
    
TUSD/USDT
CREATE TABLE TUSD_USDT( 
date DATE, 
predict varchar(255)
)
    
IOTA/USDT
CREATE TABLE IOTA_USDT( 
date DATE, 
predict varchar(255)
)
    
XLM/USDT
CREATE TABLE XLM_USDT( 
date DATE, 
predict varchar(255)
)
    
ONT/USDT
CREATE TABLE ONT_USDT( 
date DATE, 


In [33]:
def insert_db(ticker,text,today_datetime):
    conn = connect_RDS(host,port,username,password,database)
    cursor = conn.cursor()
    # 데이터 삽입
    sql = "INSERT INTO " + str(ticker).replace("/","_") + "(date,predict) VALUES ("+'"'+ str(today_datetime)+'"' +"," + '"' + str(text) + '"' + ")"
    print(sql)
    cursor.execute(sql)
    conn.commit()
    conn.close()

In [34]:
ticker = download_data().Usdt_symbol()
for i in ticker:
    flag = 0
    try:
        print(i)
        today_datetime = datetime.today().strftime("%Y-%m-%d") # YYYY-mm-dd 형태의 시간 출력
        data = download_data(since = "2022-05-01 00:00:00",code_name = str(i),frame='1d').get_data()
        data = add_ma(data)
        data = add_rsi(data)
        data = add_ema(data)
        data = add_stochastic(data)
        data = add_disparity(data)
        data = data.dropna()
        scaler = MinMaxScaler()
        try:
            df_scaled = scaler.fit_transform(data)

            df_scaled = pd.DataFrame(df_scaled)
            df_scaled.columns = data.columns
            # slicing data
            try:
                train_feature = pd.DataFrame(df_scaled.drop(['close'],axis=1))
                train_label = pd.DataFrame(df_scaled['close'])
                train_feature, train_label = make_dataset(train_feature, train_label, window_size = 20)
                try:
                    model = joblib.load('model_'+str(i).replace("/","_")+'.pkl')
                    temp = train_feature[-1].reshape(1,20,12)
                    result = model.predict(temp)[0][-1][0]
                    today = df_scaled.iloc[-1]['close']
                    if result > today:
                        insert_db(i,"상승",today_datetime)
                        flag = 1
                        print(str(i),": 상승")
                    else:
                        insert_db(i,"하락",today_datetime)
                        flag = 1
                        print(str(i),": 하락")
                except:
                    if flag == 0:
                        insert_db(i,"model predict error",today_datetime)
                        flag = 1
                        print(str(i),": model predict error")
            except:
                if flag == 0:
                    insert_db(i,"data slice error",today_datetime)
                    flag = 1
                    print(str(i),"data slice error")
        except:
            if flag == 0:
                insert_db(i,"data scale error",today_datetime)
                flag = 1
                print(str(i),": data scale error")
    except:
        if flag == 0:
            insert_db(i,"data read error",today_datetime)
            flag = 1
            print(str(i),": data read error")

BTC/USDT
INSERT INTO BTC_USDT(date,predict) VALUES ("2022-08-19","상승")
BTC/USDT : 상승
ETH/USDT
INSERT INTO ETH_USDT(date,predict) VALUES ("2022-08-19","하락")
ETH/USDT : 하락
BNB/USDT
INSERT INTO BNB_USDT(date,predict) VALUES ("2022-08-19","하락")
BNB/USDT : 하락
BCC/USDT
INSERT INTO BCC_USDT(date,predict) VALUES ("2022-08-19","data read error")
BCC/USDT : data read error
NEO/USDT
INSERT INTO NEO_USDT(date,predict) VALUES ("2022-08-19","상승")
NEO/USDT : 상승
LTC/USDT
INSERT INTO LTC_USDT(date,predict) VALUES ("2022-08-19","하락")
LTC/USDT : 하락
QTUM/USDT
INSERT INTO QTUM_USDT(date,predict) VALUES ("2022-08-19","상승")
QTUM/USDT : 상승
ADA/USDT
INSERT INTO ADA_USDT(date,predict) VALUES ("2022-08-19","상승")
ADA/USDT : 상승
XRP/USDT
INSERT INTO XRP_USDT(date,predict) VALUES ("2022-08-19","상승")
XRP/USDT : 상승
EOS/USDT
INSERT INTO EOS_USDT(date,predict) VALUES ("2022-08-19","하락")
EOS/USDT : 하락
TUSD/USDT
INSERT INTO TUSD_USDT(date,predict) VALUES ("2022-08-19","상승")
TUSD/USDT : 상승
IOTA/USDT
INSERT INTO IOTA_USDT(d